In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV as gridsearchcv
from sklearn.metrics import mean_squared_error, make_scorer, r2_score, auc, roc_curve
import sklearn.metrics
from sklearn.neural_network import MLPClassifier as mlp
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.ensemble import AdaBoostClassifier as ada
from sklearn.svm import SVC as svc

from scipy import stats

import math
import os
import random

In [ ]:
## Move to correct folder for server.  Can remove before sending
# os.chdir('/home/poblivsig/Dropbox/horses2')
os.chdir('/home/poblivsig/Dropbox/cancer_and_phishing')

print(os.getcwd())

In [ ]:
## Open the pre-processed csv
df = pd.read_csv('data/winequality-red.csv')

In [ ]:
## Get info about wine
print(f'Shape\n\n{df.shape}')
print(f'Columns\n\n{df.columns}')
print(f'dtypes\n\n{df.dtypes}')
pd.set_option('display.max_columns', None)
print(f'Description\n\n{df.describe()}')
print(f'Info:\n{df.info}')
print(f'Check out the sample: {df.sample(n=1)}')
pd.set_option('display.max_columns', 5)

In [ ]:
# Count the different quality values
sns.countplot(df['quality'],
              palette='Blues',
              label="Quality Count", )
plt.plot()
plt.savefig('data/charts/bc_diag_countplot.png')

In [ ]:
# Create mini-histograms for each attribute
df.hist(bins=10,
        figsize=(10, 8))
plt.show()


In [ ]:
# Find the amount of correlation between each column and the quality
corrs = df.corr()
corr_quality = corrs['quality']
print('Amount of correlation (Pearsons r) for each column:')
print(corr_quality.sort_values(ascending=False))

In [ ]:
# Create a scatterplot matrix with a Kernel Density estimation on the diagonal
scatter_matrix = pd.plotting.scatter_matrix(df, diagonal='kde', cmap='Blues', figsize=(12, 12))

#May need to offset label when rotating to prevent overlap of figure
[scat.get_yaxis().set_label_coords(-0.9, 0.4) for scat in scatter_matrix.reshape(-1)]

# Rotate all of the column names
[scat.xaxis.label.set_rotation(45) for scat in scatter_matrix.reshape(-1)]
[scat.yaxis.label.set_rotation(0) for scat in scatter_matrix.reshape(-1)]

# Remove all of the markings and numbers along the axes
[scat.set_xticks(()) for scat in scatter_matrix.reshape(-1)]
[scat.set_yticks(()) for scat in scatter_matrix.reshape(-1)]

plt.show()

In [ ]:
col_names = ['fixed acidity',
             'volatile acidity',
             'citric acid',
             'residual sugar',
             'chlorides',
             'free sulfur dioxide',
             'total sulfur dioxide',
             'density',
             'pH',
             'sulphates',
             'alcohol',
             'quality']
fig, ax = plt.subplots(figsize=(10, 10))

# Create color map
colormap = sns.diverging_palette(220,
                                 10,
                                 as_cmap=True)

# Create Heat Map, including annotations
# Put the floating point numbers in the map
sns.heatmap(corrs,
            cmap='Blues',
            fmt=".2f",
            annot=True)

ax.set_xticklabels(
    col_names,
    horizontalalignment='right',
    rotation=45)

ax.set_yticklabels(col_names)
plt.show()

In [ ]:
###############################
# Build boxplots for the most correlated against the different quality levels (3)

## Alcohol
plot = sns.boxplot(x='quality',
                   y='alcohol',
                   data=df,
                   showfliers=False)
plot.set_title('Quality vs. Alcohol')
plt.plot()

In [ ]:
## Volatile acidity
plot = sns.boxplot(x='quality',
                   y='volatile acidity',
                   data=df,
                   showfliers=False)
plot.set_title('Quality vs. Volatile Acidity')
plt.plot()

In [ ]:
## Sulphates
plot = sns.boxplot(x='quality',
                   y='sulphates',
                   data=df,
                   showfliers=False)
plot.set_title('Quality vs. Sulphates')
plt.plot()

In [ ]:
## Citric Acid
plot = sns.boxplot(x='quality',
                   y='citric acid',
                   data=df,
                   showfliers=False)
plot.set_title('Quality vs. Fixed Acidity')
plt.plot()

In [ ]:
## Residual Sugar
plot = sns.boxplot(x='quality',
                   y='residual sugar',
                   data=df,
                   showfliers=False)
plot.set_title('Quality vs. Residual Sugar')
plt.plot()

In [ ]:
## Split the data up.
y = df['quality']
X = df.drop('quality', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.30,
                                                    stratify=y,
                                                    random_state=14)
print(df.shape)

In [ ]:
# Scale the features (attributes)
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



# print(cross_val_score(decisiontreeclassifier(criterion = 'gini', random_state = 0), x_train, y_train, cv=5))
def gridsearch(estimator, param_grid, cv, scoring_metric):
    scorer = sklearn.metrics.make_scorer(sklearn.metrics.f1_score, average = 'weighted')
    clf = gridsearchcv(estimator=estimator,
                       param_grid=param_grid,
                       n_jobs=-1,
                       cv=cv,
                       return_train_score=True,
                       scoring=scorer,
                       verbose=3)
    clf.fit(X_train, y_train)

    ### Output the results
    print(f'Best parameters: {clf.best_params_}')
    print(f'Best score: {clf.best_score_}')
    best_estimate = clf.best_estimator_
    print(best_estimate)

    ## Now we have found the best parameters, use them...
    best_estimate.fit(X_train,y_train)

    predictor = best_estimate.predict(X_train)
    mse = mean_squared_error(predictor, y_train)
    r2 = r2_score(predictor, y_train)
    print(f'Training Mean Square Error: {mse:.2f}')
    print(f'Training R2: {r2:.2f}')

    y_predictor = best_estimate.predict(X_test)
    mse = mean_squared_error(y_test, y_predictor)
    r2 = r2_score(y_test, y_predictor)
    print(f'Testing Mean Square Error: {mse:.2f}')
    print(f'Testing R2: {r2:.2f}')
    print('blah')

    return best_estimate, y_predictor


In [ ]:
def cm_and_class_rep(X_test, y_test, y_predictor, best_estimate):
    confusion_matrix(y_test, y_predictor)
    cm = plot_confusion_matrix(best_estimate,
                               X_test,
                               y_test,
                               cmap=plt.cm.Blues,
                               normalize='true' )
    plt.show(cm)
    plt.show()
    print(classification_report(y_test, y_predictor))

In [ ]:
def draw_learning_curve(estimator_1,
                        estimator_1_name,
                        estimator_2,
                        estimator_2_name,
                        estimator_3,
                        estimator_3_name,
                        X_train,
                        y_train,
                        cv,
                        train_max,
                        title):

    # Set plot size
    plt.figure(figsize=(7,5))

    ###################################
    # Do the 1st curve
    sizes, \
    training_scores, \
    testing_scores, \
    fit_times, \
    score_times = learning_curve(estimator_1,
                                 X_train,
                                 y_train,
                                 cv=cv,
                                 scoring='accuracy',
                                 return_times=True,
                                 train_sizes=np.arange(1, train_max, 10))

    # Mean of training scores
    mean_training = np.mean(training_scores, axis=1)

    # Mean of testing scores
    mean_testing = np.mean(testing_scores, axis=1)

    # Do the best lines
    plt.plot(sizes,
             mean_training,
             '--',
             label='Training Score - ' + estimator_1_name,
             color='blue')
    plt.plot(sizes,
             mean_testing,
             label='Cross Validation Score - ' + estimator_1_name,
             color='cornflowerblue')


    ###################################
    # Do the 2nd curve
    sizes, \
    training_scores, \
    testing_scores, \
    fit_times, \
    score_times = learning_curve(estimator_2,
                                 X_train,
                                 y_train,
                                 cv=cv,
                                 scoring='accuracy',
                                 return_times=True,
                                 train_sizes=np.arange(1, train_max, 10))

    # Mean of training scores
    mean_training = np.mean(training_scores, axis=1)

    # Mean of testing scores
    mean_testing = np.mean(testing_scores, axis=1)

    # Do the best lines
    plt.plot(sizes,
             mean_training,
             '--',
             label='Training Score - ' + estimator_2_name,
             color='green')
    plt.plot(sizes,
             mean_testing,
             label='Cross Validation Score - ' + estimator_2_name,
             color='springgreen')

    ###################################
    # Do the 3rd curve
    sizes, \
    training_scores, \
    testing_scores, \
    fit_times, \
    score_times = learning_curve(estimator_3,
                                 X_train,
                                 y_train,
                                 cv=cv,
                                 scoring='accuracy',
                                 return_times=True,
                                 train_sizes=np.arange(1, train_max, 10))

    # Mean of training scores
    mean_training = np.mean(training_scores, axis=1)

    # Mean of testing scores
    mean_testing = np.mean(testing_scores, axis=1)

    # Do the best lines
    plt.plot(sizes,
             mean_training,
             '--',
             label='Training Score - ' + estimator_3_name,
             color='red')
    plt.plot(sizes,
             mean_testing,
             label='Cross Validation Score - ' + estimator_3_name,
             color='lightcoral')

    # Do the final plots
    plt.title(title)
    plt.xlabel('Training Set Size'), plt.ylabel('Accuracy'), plt.legend(loc="best")
    plt.tight_layout()
    plt.legend(loc="best")
    plt.savefig('data/charts/wine_learning_curve.png')
    plt.show()

In [ ]:
# Draw Validation Curve
def draw_validation_curve(estimator, X_train, y_train, cv, param_name, param_range, title, xlabel):

    train_scores, test_scores = validation_curve(estimator,
                                                 X_train,
                                                 y_train,
                                                 param_name=param_name,
                                                 param_range=param_range,
                                                 cv=cv,
                                                 scoring='accuracy',
                                                 n_jobs=-1)

    # Mean from the scores for the training set
    train_mean = np.mean(train_scores, axis=1)

    # Mean from the scores for the test set
    test_mean = np.mean(test_scores, axis=1)

    # Create plot for training
    plt.plot(param_range,
             train_mean,
             label="Training score",
             color="blue")

    # Create plot for testing
    plt.plot(param_range,
             test_mean,
             label="Cross-validation score",
             color="red")

    # Build plot
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Accuracy Score")
    plt.legend(loc="best")
    plt.tight_layout()
    plt.savefig('data/charts/wine_validation_curve.png')
    plt.show()

In [ ]:
# Do Decision Tree stuff...

# Do a grid search for the Decision Tree
dt_criterion = ['gini', 'entropy']
dt_max_depth = [9] #[count for count in range(0, 30)]
dt_class_weight = [None, 'balanced']
print(f'dt_max_depth = {dt_max_depth}')
param_grid = dict(criterion=dt_criterion,
                  max_depth=dt_max_depth,
                  class_weight=dt_class_weight)

best_estimate, y_predictor = gridsearch(estimator=dtc(),
                                        param_grid=param_grid,
                                        cv=8,
                                        scoring_metric='recall')

cm_and_class_rep(X_test, y_test, y_predictor, best_estimate)

In [ ]:
# Decision Tree learning curve

## TODO: Grab these values from best_estimate above
tuned_criterion = 'entropy'
tuned_max_depth=9
tuned_class_weight='balanced'

# Best depth
draw_learning_curve(dtc(criterion=tuned_criterion,
                        max_depth=10,
                        class_weight=tuned_class_weight),
                    'max_depth = 10',
                    dtc(criterion=tuned_criterion,
                        max_depth=17,
                        class_weight=tuned_class_weight),
                    'max_depth = 17',
                    dtc(criterion=tuned_criterion,
                        max_depth=24,
                        class_weight=tuned_class_weight),
                    'max_depth = 24',
                    X_train,
                    y_train,
                    cv=8,
                    train_max=900,
                    title = 'DT Red Wine Learning Curve Training Set Size vs Accuracy using Various Max Depths')

In [ ]:
# Draw Decision Tree Validation Curve
draw_validation_curve(dtc(criterion=tuned_criterion,
                          class_weight=tuned_class_weight),
                      X_train,
                      y_train,
                      cv=8,
                      param_name='max_depth',
                      param_range=np.arange(0,80),
                      title='DT Red Wine Validation Curve for Maximum Depth',
                      xlabel='Maximum Depth')

In [ ]:
# Do KNN stuff
# Set the parameters by cross-validation
MAX_NEIGHBORS=19
n_neighbors=[layers for layers in range(1, MAX_NEIGHBORS)]
metric=['manhattan', 'euclidean', 'chebyshev']
weights=['uniform', 'distance']
algorithm=['kd_tree', 'ball_tree']
param_grid = dict(n_neighbors=n_neighbors,
                  metric=metric,
                  weights=weights,
                  algorithm=algorithm)
best_estimate, y_predictor = gridsearch(estimator=knn(),
                                        param_grid=param_grid,
                                        cv=8,
                                        scoring_metric='precision')

cm_and_class_rep(X_test, y_test, y_predictor, best_estimate)


In [ ]:
# KNN learning curve
tuned_n_neighbors = 13
tuned_metric = 'manhattan'
tuned_weights = 'distance'
tuned_algorithm = 'kd_tree'

draw_learning_curve(knn(n_neighbors=3,
                        metric=tuned_metric,
                        weights=tuned_weights,
                        algorithm=tuned_algorithm),
                    'Nearest Neighbors = 2',
                    knn(n_neighbors=4,
                        metric=tuned_metric,
                        weights=tuned_weights,
                        algorithm=tuned_algorithm),
                    'Nearest Neighbors = 3',
                    knn(n_neighbors=5,
                        metric=tuned_metric,
                        weights=tuned_weights,
                        algorithm=tuned_algorithm),
                    'Nearest Neighbors = 4',
                    X_train,
                    y_train,
                    cv=8,
                    train_max=500,
                    title = 'KNN Red Wine Learning Curve Training Set Size vs Nearest Neighbors')

In [ ]:
tuned_n_neighbors = 13
tuned_metric = 'manhattan'
tuned_weights = 'distance'
tuned_algorithm = 'kd_tree'

# Draw KNN Validation Curve
draw_validation_curve(knn(n_neighbors=tuned_n_neighbors,
                          metric=tuned_metric,
                          weights=tuned_weights,
                          algorithm=tuned_algorithm),
                      X_train,
                      y_train,
                      cv=7,
                      param_name='n_neighbors',
                      param_range=np.arange(0, MAX_NEIGHBORS),
                      title='KNN Red Wine Validation for Nearest Neighbors',
                      xlabel='Number of Nearest Neighbors')

In [ ]:
# Do Boosting stuff
# Set the parameters by cross-validation
ada_learning_rate = [(2**x)/100 for x in range(7)]
# ada_n_estimators = [1, 2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
ada_n_estimators = [20, 30, 40]
# N.B. These hyper-parameters are from previous decision tree tuning.
ada_dt = dtc(criterion=tuned_criterion,
             max_depth=tuned_max_depth,
             class_weight=tuned_class_weight)I
param_grid = dict(learning_rate=ada_learning_rate,
                  n_estimators=ada_n_estimators)
best_estimate, y_predictor = gridsearch(estimator=ada(ada_dt),
                                        param_grid=param_grid,
                                        cv=10,
                                        scoring_metric='recall')

cm_and_class_rep(X_test, y_test, y_predictor, best_estimate)

In [ ]:
# Boosting learning curve

## Grab these values from best_estimate above
tuned_learning_rate = 0.16
tuned_n_estimators = 80

draw_learning_curve(ada(ada_dt,
                        learning_rate=tuned_learning_rate,
                        n_estimators=20),
                    'no. of estimators = 20',
                    ada(ada_dt,
                        learning_rate=tuned_learning_rate,
                        n_estimators=80),
                    'no. of estimators = 80',
                    ada(ada_dt,
                        learning_rate=tuned_learning_rate,
                        n_estimators=140),
                    'no. of estimators = 140',
                    X_train,
                    y_train,
                    cv=8,
                    train_max=500,
                    title = 'AdaBoost Red Wine Learning Curve Training Set Size vs No. of Estimators')

In [ ]:
# Draw Boosting Validation Curve
draw_validation_curve(ada(ada_dt,
                          learning_rate=tuned_learning_rate,
                          n_estimators=tuned_n_estimators),
                      X_train,
                      y_train,
                      cv=8,
                      param_name='n_estimators',
                      param_range=ada_n_estimators,
                      title='AdaBoost Red Wine Validation Curve for No. of Estimators',
                      xlabel='Number of Estimators')

In [ ]:
# Do SVM stuff
# Set the parameters by cross-validation
svc_C = np.arange(0.001, 2.5, 0.5)
svc_tol = np.arange(1e-8, 1e-1, 0.05)
svc_kernel = ['linear', 'poly', 'rbf', 'sigmoid']
# svc_gamma = [1, 0.1, 0.01, 0.001, 0.0001]
svc_gamma = [0.1]
param_grid = dict(C=svc_C,
                  tol=svc_tol,
                  kernel=svc_kernel,
                  gamma = svc_gamma)
best_estimate, y_predictor = gridsearch(estimator=svc(),
                                        param_grid=param_grid,
                                        cv=8,
                                        scoring_metric='precision')

cm_and_class_rep(X_test, y_test, y_predictor, best_estimate)

In [ ]:
tuned_C = 1.501
tuned_tol = 1 ** -8
tuned_kernel = 'rbf'
tuned_gamma = 1

# SVM learning curve
draw_learning_curve(svc(C=tuned_C,
                        tol=tuned_tol,
                        kernel=tuned_kernel,
                        gamma=0.1),
                        'gamma = 0.1',
                    svc(C=tuned_C,
                        tol=tuned_tol,
                        kernel=tuned_kernel,
                        gamma=1),
                     'gamma = 1',
                    svc(C=tuned_C,
                        tol=tuned_tol,
                        kernel=tuned_kernel,
                        gamma=10),
                    'gamma = 10',
                    X_train,
                    y_train,
                    cv=8,
                    train_max=500,
                    title = 'SVM Red Wine Learning Curve Training Set Sizes vs Gamma')

In [ ]:
# Draw SVM Validation Curve
draw_validation_curve(svc(C=tuned_C,
                          tol=tuned_tol,
                          kernel=tuned_kernel,
                          gamma=1),
                      X_train,
                      y_train,
                      cv=8,
                      param_name='gamma',
                      param_range=[10, 1, 0.1, 0.01, 0.001, 0.0001],
                      title='SVM Red Wine Validation for Gamma',
                      xlabel='Gamma')

In [ ]:
# Do MLP stuff...
MAX_LAYER_SIZE = 20
hidden_layer_sizes = [layers for layers in range(1, MAX_LAYER_SIZE)]
activation = ['tanh', 'relu']
max_iter = [500, 1000, 1500, 2000]
alpha = 10.0 ** -np.arange(1,7)
learning_rate = ['constant', 'adaptive']
param_grid = dict(hidden_layer_sizes=hidden_layer_sizes,
                  activation=activation,
                  max_iter=max_iter,
                  alpha=alpha,
                  learning_rate=learning_rate)

best_estimate, y_predictor = gridsearch(estimator=mlp(),
                                        param_grid=param_grid,
                                        cv=8,
                                        scoring_metric='precision')

cm_and_class_rep(X_test, y_test, y_predictor, best_estimate)


# MLP learning curve
tuned_hidden_layer_sizes = 19
tuned_activation = 'tanh'
tuned_max_iter = 1000
tuned_alpha = 1e-06
tuned_learning_rate = 'constant'

draw_learning_curve(mlp(hidden_layer_sizes=4,
                        activation=tuned_activation,
                        max_iter=tuned_max_iter,
                        alpha=tuned_alpha,
                        learning_rate=tuned_learning_rate),
                    'hidden_layer_size = 14',
                    mlp(hidden_layer_sizes=tuned_hidden_layer_sizes,
                        activation=tuned_activation,
                        max_iter=tuned_max_iter,
                        alpha=tuned_alpha,
                        learning_rate=tuned_learning_rate),
                    'hidden_layer_size = 19',
                    mlp(hidden_layer_sizes=34,
                        activation=tuned_activation,
                        max_iter=tuned_max_iter,
                        alpha=tuned_alpha,
                        learning_rate=tuned_learning_rate),
                    'hidden_layer_size = 24',
                    X_train,
                    y_train,
                    cv=8,
                    train_max=500,
                    title = 'MLP Phishing Learning Curve Training Set Size vs. hidden layer size')


In [ ]:
# Draw MLP Validation Curve
draw_validation_curve(mlp(hidden_layer_sizes=tuned_hidden_layer_sizes,
                          activation=tuned_activation,
                          max_iter=tuned_max_iter,
                          alpha=tuned_alpha,
                          learning_rate=tuned_learning_rate),
                      X_train,
                      y_train,
                      cv=8,
                      param_name='hidden_layer_sizes',
                      param_range=np.arange(0, MAX_LAYER_SIZE),
                      title='MLP Phishing Validation Curve for Hidden Layer Sizes',
                      xlabel='Hidden Layer Sizes')



